In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("QSTop10.csv")

In [3]:
for col in ['switch', 'line', 'slowmo', 'ig', 'replayed']:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
for col in ['trick', 'obstacle', 'obstacledetail', 'location', 'notes']:
    df[col] = df[col].str.lower()

In [4]:
# df

In [5]:
# total num_tricks
num_tricks = pd.DataFrame(df.groupby('week')['rank'].max()).reset_index()
num_tricks.columns = ['week', 'num_tricks']

# total slowmo clips
num_slowmo = pd.DataFrame(df.groupby('week').sum()['slowmo']).reset_index()
num_slowmo.columns = ['week', 'num_slowmo']

# total switch/nollie tricks
num_switch = pd.DataFrame(df.groupby('week').sum()['switch']).reset_index()
num_switch.columns = ['week', 'num_switch']

# total tricks part of line
num_line = pd.DataFrame(df.groupby('week').sum()['line']).reset_index()
num_line.columns = ['week', 'num_line']

# total IG tricks
num_ig = pd.DataFrame(df.groupby('week').sum()['ig']).reset_index()
num_ig.columns = ['week', 'num_ig']

# total replayed tricks
num_replayed = pd.DataFrame(df.groupby('week').sum()['replayed']).reset_index()
num_replayed.columns = ['week', 'num_replayed']

# total fs tricks
num_fs = pd.DataFrame(df[['trick', 'week', 'obstacle']].drop_duplicates().groupby(['week'])['trick'].apply(lambda x: x[x.str.contains('fs ')].count())).reset_index()
num_fs.columns = ['week', 'num_fs']

# total bs tricks
num_bs = pd.DataFrame(df[['trick', 'week', 'obstacle']].drop_duplicates().groupby(['week'])['trick'].apply(lambda x: x[x.str.contains('bs ')].count())).reset_index()
num_bs.columns = ['week', 'num_bs']

In [6]:
df_stats = df.merge(num_tricks).merge(num_slowmo).merge(num_switch).merge(num_line).merge(num_ig).merge(num_replayed).merge(num_fs).merge(num_bs)

In [7]:
weeks = df.groupby('week')

In [8]:
# percent stair
perc_stair = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='stairs'])) / len(x)))
perc_stair = perc_stair.reset_index()
perc_stair.columns = ['week', 'perc_stair']
# print("Stairs: {}".format(sum(perc_stair['perc_stair'])/perc_stair['week'].max()))

# percent gap
perc_gap = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='gap'])) / len(x)))
perc_gap = perc_gap.reset_index()
perc_gap.columns = ['week', 'perc_gap']

# percent ledge
perc_ledge = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='ledge'])) / len(x)))
perc_ledge = perc_ledge.reset_index()
perc_ledge.columns = ['week', 'perc_ledge']

# percent manual
perc_manual = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='pad']) + len(x[x=='manual']) ) / len(x)))
perc_manual = perc_manual.reset_index()
perc_manual.columns = ['week', 'perc_manual']

# percent transition
perc_transition = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='transistion'])) / len(x))) # yeah, I mispelled it in dataset
perc_transition = perc_transition.reset_index()
perc_transition.columns = ['week', 'perc_transition']

# percent flat
perc_flat = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='flat'])) / len(x)))
perc_flat = perc_flat.reset_index()
perc_flat.columns = ['week', 'perc_flat']

# percent hubba
perc_hubba = pd.DataFrame(weeks['obstacledetail'].agg(lambda x: (len(x[x=='hubba'])) / len(x)))
perc_hubba = perc_hubba.reset_index()
perc_hubba.columns = ['week', 'perc_hubba']

# percent handrail
perc_handrail = pd.DataFrame(weeks['obstacledetail'].agg(lambda x: (len(x[x==r'handrail'])) / len(x)))
perc_handrail = perc_handrail.reset_index()
perc_handrail.columns = ['week', 'perc_handrail']

# percent curb
perc_curb = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='curb'])) / len(x)))
perc_curb = perc_curb.reset_index()
perc_curb.columns = ['week', 'perc_curb']

# percent Wall
perc_wall = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='wall'])) / len(x)))
perc_wall = perc_wall.reset_index()
perc_wall.columns = ['week', 'perc_wall']

# percent Rail
perc_rail = pd.DataFrame(weeks['obstacle'].agg(lambda x: (len(x[x=='rail'])) / len(x)))
perc_rail = perc_rail.reset_index()
perc_rail.columns = ['week', 'perc_rail']

In [9]:
new_df = df
for s in [num_tricks, num_slowmo, num_switch, num_fs, num_bs, num_replayed, num_ig, num_line,
          perc_stair, perc_gap, perc_ledge, perc_manual, perc_transition, perc_flat, perc_hubba,
          perc_rail, perc_handrail, perc_curb, perc_wall]:
    new_df = new_df.merge(s)

cols_to_keep = ['week', 'num_tricks', 'num_slowmo', 'num_switch',
                'num_fs', 'num_bs', 'num_replayed', 'num_ig', 'num_line',
                'perc_stair', 'perc_gap', 'perc_ledge', 'perc_manual', 'perc_transition',
                'perc_hubba', 'perc_rail', 'perc_handrail', 'perc_curb', 'perc_flat', 'perc_wall']

In [10]:
stats_df = new_df[cols_to_keep].drop_duplicates().reset_index().drop(columns='index')

In [11]:
stats_df['perc_slowmo'] = stats_df.apply(lambda x: x['num_slowmo'] / x['num_tricks'], axis=1)

stats_df['perc_switch'] = stats_df.apply(lambda x: x['num_switch'] / x['num_tricks'], axis=1)

stats_df['perc_fs'] = stats_df.apply(lambda x: x['num_fs'] / x['num_tricks'], axis=1)

stats_df['perc_bs'] = stats_df.apply(lambda x: x['num_bs'] / x['num_tricks'], axis=1)

stats_df['perc_replayed'] = stats_df.apply(lambda x: x['num_replayed'] / x['num_tricks'], axis=1)

stats_df['perc_ig'] = stats_df.apply(lambda x: x['num_ig'] / x['num_tricks'], axis=1)

stats_df['perc_line'] = stats_df.apply(lambda x: x['num_line'] / x['num_tricks'], axis=1)

stats_df = stats_df.drop(columns=['num_slowmo', 'num_switch',
                                  'num_fs', 'num_bs', 'num_replayed', 'num_ig', 'num_line'])

In [12]:
for col in ['perc_stair', 'perc_gap', 'perc_ledge', 'perc_manual', 'perc_transition',
            'perc_hubba', 'perc_rail', 'perc_handrail', 'perc_curb', 'perc_flat',
            'perc_slowmo', 'perc_switch', 'perc_fs', 'perc_bs', 'perc_wall', 'perc_replayed',
            'perc_ig', 'perc_line']:
    stats_df[col] = round(stats_df[col] * 100)

In [13]:
average_percs = round(stats_df.sum()/stats_df['week'].max(), 1)

In [14]:
#REGEX FUNCTIONS
def skatercount(regex):
    return df[df['skater'].str.contains(regex, na=False)].groupby('skater')['skater']


def trickgroup(regex):
    return df[df['trick'].str.contains(regex, na=False)].groupby('trick')['trick']


def spotfind(regex):
    return df[df['location'].str.contains(regex, na=False)].groupby('location')['location']


def obstaclecount1(regex):
    return df[df['obstacle'].str.contains(regex, na=False)].groupby('obstacle')['obstacle']


def obstacledetailcount(regex):
    return df[df['obstacledetail'].str.contains(regex, na=False)].groupby('obstacledetail')['obstacledetail']


def noted(regex):
    return df[df['notes'].str.contains(regex, na=False)].groupby('notes')['notes']

# PERCENTAGES

In [15]:
average_percs

week               25.0
num_tricks         10.0
perc_stair          8.6
perc_gap           20.4
perc_ledge         32.0
perc_manual         7.6
perc_transition     4.5
perc_hubba          5.5
perc_rail          12.0
perc_handrail       5.9
perc_curb           2.9
perc_flat           4.1
perc_wall           4.5
perc_slowmo        33.3
perc_switch        16.3
perc_fs            29.2
perc_bs            51.0
perc_replayed      36.1
perc_ig            21.6
perc_line          28.0
dtype: float64

##### Numbers of Slowmo, Switch/Nollie etc

In [16]:
for s in [num_tricks, num_slowmo, num_switch, num_fs, num_bs, num_replayed, num_ig, num_line]:
    num_df = new_df.merge(s)
num_df = num_df[['week', 'num_slowmo', 'num_switch', 'num_replayed', 'num_ig', 
                        'num_line']].drop_duplicates().reset_index().drop(columns='index')
num_df.sum()

week            1225.0
num_slowmo       163.0
num_switch        80.0
num_replayed     177.0
num_ig           106.0
num_line         137.0
dtype: float64

In [17]:
total_points = pd.DataFrame(df.groupby('skater')['points'].sum()).reset_index()
total_points.columns = ['skater', 'total_points']
top_points = total_points.sort_values(['total_points'], ascending=False)

# TOP SKATERS By POINTS

In [18]:
top_points.head(35)
# top_points.head(35).to_csv('/Users/peteglover/Desktop/QS_points.csv')

,skater,total_points
218,Mason Silva,55
202,Lucas Puig,42
327,Tom Knox,38
321,Tiago Lemos,35
16,Alexis Ramirez,33
135,Jacopo Carozzi,30
66,Cyrus Bennett,29
266,Pedro Delfino,29
248,Nik Stain,29
24,Antonio Durao,26


# TOP SKATERS by APPEARANCES

In [19]:
appearances = skatercount(r"").count().sort_values(ascending=False)
appearances.head(35)

skater
Lucas Puig             6
Mason Silva            6
Tom Knox               5
Will Marshall          5
Tiago Lemos            4
Louie Lopez            4
Hyun Kummer            4
Carl Aikens            4
Antonio Durao          4
Jacopo Carozzi         4
Alexis Ramirez         4
Nik Stain              4
Cyrus Bennett          4
John Shanahan          4
Mike Arnold            3
Milton Martinez        3
Oskar Rozenberg        3
Pedro Attenborough     3
Martino Cattaneo       3
Pedro Delfino          3
Giovanni Vianna        3
Marcello Campanello    3
Gustav Tonnesen        3
Remy Taveira           3
Max Palmer             3
Casper Brooker         3
Alexis Lacroix         3
Ish Cepeda             3
Karsten Kleppan        3
Ville Wester           3
Jaakko Ojanen          3
Tyshawn Jones          3
Zered                  3
Hjalte Halberg         3
Rob Maatman            2
Name: skater, dtype: int64

In [20]:
yes = [i for i in appearances.values if i > 3]
len(yes)

14

In [21]:
print("Unique skaters in #QSTop10 2020: {}".format(len(skatercount(r""))))

Unique skaters in #QSTop10 2020: 354


# Total Number of TRICKS

In [22]:
total_tricks = num_tricks['num_tricks'].sum()

In [23]:
total_tricks

490

# Number of UNIQUE TRICKS

In [24]:
unique_tricks = []
for i in df['trick']:
    if i not in unique_tricks:
        unique_tricks.append(i)

In [25]:
len(unique_tricks)

370

In [26]:
print('Percentage Tricks that are Unique: {}'.format(round(len(unique_tricks)/total_tricks*100, 1)))

Percentage Tricks that are Unique: 75.5


# TOP TRICKS

In [27]:
toptricks = trickgroup(r"").count().sort_values(ascending=False)
toptricks.head(25)

trick
ollie                              16
kickflip                           15
360flip                            11
bs kickflip                         6
bs boardslide                       5
hardflip                            5
switch hardflip                     5
bs lipslide                         5
fs 50-50                            5
switch kickflip                     4
bs 360 ollie                        4
fs lipslide                         4
bs nosebluntslide                   4
bs 50-50                            4
bs noseslide                        4
heelflip                            3
switch fs kickflip                  3
switch bs shove                     3
bs wallride                         3
switch fs crooked                   3
bs noseslide to nollie kickflip     3
nollie bs heelflip                  2
nollie fs 360                       2
bs bluntslide                       2
bs caballerial                      2
Name: trick, dtype: int64

In [28]:
yes1 = [i for i in toptricks.values if i == 2] #i == number of occurances
# 26 tricks happened twice
len(yes1)

26

## Trickfinder

In [29]:
trickfind = trickgroup(r"slap").count().sort_values(ascending=False)
trickfind

trick
switch slappy fs crooked to fakie kickflip        1
slappy ollie to bs nosebluntslide                 1
slappy fs crooked to nollie bs kickflip           1
slappy fs crooked                                 1
slappy bs smith                                   1
slappy bs nosebluntslide                          1
slappy bs 50-50                                   1
fs slappy nosegrind                               1
fakie slappy to fakie manual to fakie kickflip    1
Name: trick, dtype: int64

In [30]:
yes2 = [i for i in trickfind.values]
sum(yes2)

9

# OBSTACLES

In [31]:
obstaclecount1(r"").count().sort_values(ascending=False)

obstacle
ledge          157
gap            100
rail            59
stairs          42
pad             37
wall            22
transistion     22
flat            20
curb            14
hip              8
bank             6
pole             3
Name: obstacle, dtype: int64

# OBSTACLE DETAILS

In [32]:
over = obstacledetailcount(r"over").count().sort_values(ascending=False) #for "over" obstacles
over

obstacledetail
over rail                 22
over barrier              10
over rail into bank        8
over trash can             7
over bench                 3
over wall                  2
over trash can to curb     2
over hydrant               2
over fence                 2
bump to over bar           1
Name: obstacledetail, dtype: int64

In [33]:
sum(i for i in over)

59

## Obstacle Detail Finder

In [34]:
details = obstacledetailcount(r"up").count().sort_values(ascending=False) #enter whatever detail to track
details.head(40)

obstacledetail
up stairs            11
wall to up stairs     1
upwards pad           1
up ledge              1
up gap                1
Name: obstacledetail, dtype: int64

In [35]:
sum(i for i in details)

15

# Top Numbers BY SKATER (IG, Switch, etc.)

In [36]:
byskater = df.groupby('skater').sum()
byskater = byskater.sort_values('replayed', ascending=False) # sorts by column HERE
byskater = byskater.drop(columns=['week', 'rank', 'points'])
byskater.head(10)

,line,switch,slowmo,ig,replayed
skater,,,,,
Mason Silva,0.0,0.0,3.0,2.0,5.0
Alexis Ramirez,0.0,0.0,3.0,1.0,3.0
Giovanni Vianna,0.0,0.0,3.0,0.0,3.0
Louie Lopez,3.0,0.0,1.0,0.0,3.0
Lucas Puig,1.0,6.0,5.0,4.0,3.0
Cyrus Bennett,0.0,0.0,0.0,2.0,2.0
Jack O’Grady,0.0,0.0,2.0,0.0,2.0
Tyshawn Jones,0.0,1.0,2.0,3.0,2.0
Oskar Rozenberg,2.0,0.0,0.0,3.0,2.0


# Top Numbers by WEEK (ig, slowmo, etc.)

In [37]:
byweek = df.groupby('week').sum()
byweek = byweek.sort_values('ig', ascending=True) # sorts by column, change ascending to get lowest
byweek = byweek.drop(columns=['rank', 'points'])
byweek.head(10)

,line,switch,slowmo,ig,replayed
week,,,,,
49,3.0,2.0,2.0,0.0,4.0
47,3.0,1.0,4.0,0.0,7.0
46,1.0,4.0,6.0,0.0,7.0
44,6.0,2.0,7.0,0.0,4.0
29,4.0,1.0,2.0,0.0,1.0
16,3.0,0.0,6.0,0.0,6.0
14,4.0,2.0,5.0,0.0,6.0
43,4.0,1.0,5.0,1.0,5.0
40,3.0,3.0,4.0,1.0,4.0


# Top SPOTS

In [38]:
spotfind(r"").count().sort_values(ascending=False)

location
skatepark                    18
sf hills                      8
diy park                      7
milan                         7
pulaski                       5
museum of natural history     5
indoor park                   4
art sculpture                 4
blubba                        4
majorca                       3
oakland courthouse            3
zuccotti park                 3
south bank                    3
macba                         2
round and round planter       2
3rd & army                    2
union square sf               2
stalin plaza                  2
staples center                2
tekashi ten                   2
emb                           1
columbus park                 1
clipper                       1
citi field benches            1
china banks                   1
chain                         1
big spool                     1
bronx courthouse              1
cbs                           1
cardiel ledge, sf             1
ft. miley                     1

# NOTES

In [39]:
noted(r"").count().sort_values(ascending=False)

notes
quick footed                                27
crazy obstacle                              26
combo                                       14
5 trick line                                 8
woman skater                                 6
6 trick line                                 5
                                             4
on novelty board                             3
trans skater                                 3
7 trick line                                 2
wrong name in titles, corrected              2
4 kickflip line                              1
5 kickflip line                              1
berrics                                      1
3 different tricks to #1                     1
3 360flip line                               1
8 trick line                                 1
aka hank scorsese                            1
wrong name in titles.                        1
black history top ten on 6/5/2020            1
wrong name in titles, corrected : tow in     1
first q